<a href="https://colab.research.google.com/github/coldsober-irene/hackathon1/blob/main/Copy_of_hackathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install  -U easynmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 24.5 MB/s 
     |████████████████████████████████| 1.3 MB 49.5 MB/s 
     |████████████████████████████████| 68 kB 6.9 MB/s 
     |████████████████████████████████| 7.6 MB 11.1 MB/s 
     |████████████████████████████████| 163 kB 14.5 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19918 sha256=e0f1c8bec03bb1e6989ba740b922f3f7505293a756449b4400e53bcd11129f6c
  Stored in directory: /root/.cache/pip/wheels/d1/57/06/53ca38645e14d4537a41e5a36da2026e10f54ae88240dd5190
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3162430 sha256=e85ba39b08de3fa4741968e26eaae4e369e28e75f2077ee97dc006605957a942
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built easy

In [ ]:
# REQUIRED IMPORTS
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import numpy as np
from easynmt import EasyNMT
import json # to access languages codes

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **WEBSCRAPING SECTION** %%%%%%%%%%%%%%%%%%%%%

In [3]:
pd.set_option('display.max_columns', None)

In [22]:
# DATA MINING
class dataMining:
  def __init__(self):
    pass
    
  # SOUP
  def Soup(self, url):
    return bs(requests.get(url).content, "html.parser")
  
  def get_text(self):
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    parent = "https://www.jobinrwanda.com"
    co_links = {}
    app_link = []
    full_text = [[tag for tag in dv.children] for j_link in [job_link for job_link in [parent+link['href'] if '/job/' in link['href'] else co_links.setdefault(link.text, parent+link['href']) 
    for div in soup.find_all('div', class_ = 'card-body p-2') for link in div.find_all('a')] if '/job/' in job_link]
    for dv in self.Soup(j_link).find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
    ]
    print(full_text)

  def get_links(self):
    # GET SOUP OBJECT
    # soup = self.Soup(url = "https://www.jobinrwanda.com/jobs/all")
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    # GET ALL JOBS LINKS
    company_infoLink = []
    
    all_jobsLinks = ["https://www.jobinrwanda.com"+link['href'] if "job" in link['href'] 
                     else company_infoLink.append(link.text) 
                     for div in soup.find_all("div", class_= "card-body p-2") 
                     for link in div.find_all("a")
                     ]
    all_jobsLinks = [link for link in all_jobsLinks if link]
    return (all_jobsLinks, company_infoLink)

  def get_jobInfo(self):
    jobLinks = self.get_links()[0]
    info = {}
    # REFINE THE INFO INTO A DICTIONARY
    def refine_info(ls):
      refined_info = {}
      infoZeroPattern = re.compile("\d+.*")
      views = infoZeroPattern.search(ls[0]).group()
      refined_info["views"] = views
      # OTHER DETAILS EXCEPT THE ONE ON POSITION 0
      otherInfoPattern = re.compile(":")
      for detail in ls[1:]:
        if detail.lower() == "apply":
          pass
        else:
          try:
            detailed = otherInfoPattern.split(detail)
            refined_info[detailed[0]] = detailed[1]
          except IndexError:
            pass
      return refined_info
       
    for index,url in enumerate(jobLinks):
      soup = self.Soup(url)
      pattern = re.compile("\s{2,}")
      job_info = [pattern.sub(" ",li.text.strip().replace("\n", "")) for ul in soup.find_all('ul', class_ = "list-group list-group-flush") 
      for li in ul.find_all('li')]
      # DICTIONANRY OF INFO
      info[index] = refine_info(job_info[:9])
    return info

  def get_description(self):
    jobLinks = self.get_links()[0]
    all_text= {}
    for index,link in enumerate(jobLinks):
      soup = self.Soup(link)
      # JOB TITLE
      job_title = soup.find('span', class_ = "field field--name-title field--type-string field--label-hidden")
      
      tags_content = soup.find_all('div', class_= "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
      
      for div in tags_content:
        temp = []
        try:
          # GET APPLICATION LINK
          app_link = soup.find("a", class_ = "btn btn-sm btn-success")['href']
          if app_link.startswith("/"):
            app_link = "https://www.jobinrwanda.com"+app_link
          temp.append(app_link)
          
        except TypeError:
          temp.append(np.nan)
        for tag in div.children:
          try:
            temp.append(tag.get_text())
          except (AttributeError, TypeError):
            pass
        temp.insert(0, job_title.text)
        all_text[index] = temp
    return all_text


In [ ]:
# CLASS OBJECT CREATION
jobInRwanda = dataMining()
job_text = jobInRwanda.get_text()

In [ ]:
print(job_text[0])

In [7]:
# MERGE JOB DESCRIPTION WITH THE JOB INFO
def final_textDemo():
  global info
  # CLASS OBJECT CREATION
  jobInRwanda = dataMining()
  job_text = jobInRwanda.get_description()
  info = jobInRwanda.get_jobInfo()

  final_demo = []
  for index in list(info.keys()):
    info_value = list(info[index].values())
    # ADD COMPANY HIRING
    co_info = jobInRwanda.get_links()[1]
    info_value.append(co_info[index])
    # INCLUDE JOB TITLE
    info_value.append(job_text[index][0])
    # INCLUDE APPLY LINK
    info_value.append(job_text[index][1])
    # JOIN PARAGRAPHS 
    t = '\n'.join(job_text[index][2:])
    g = re.compile("\n|\\xa0").sub("", t)
    info_value.append(g)
    final_demo.append(info_value)
    
  return final_demo

In [11]:
job_details = final_textDemo()

In [12]:
def makeDataFrame(dataList, columns):
  return pd.DataFrame(dataList, columns = columns)

In [13]:
# DATAFRAME FORMED
titles = list(info[0].keys()) + ["Company Hiring",'Job Title', "Application link", "Job Description"]
df = makeDataFrame(final_textDemo(), titles)


In [14]:
df.tail(30)

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Company Hiring,Job Title,Application link,Job Description
53,535 times,"Kigali , RW","Agriculture, Agronomy, Business, Human resour...",Bachelor,Entry level (1 to 3 years of experience),Full-time,"Wednesday, 28/12/2022 23",1,One Acre Fund,Global HR Support Specialist,https://grnh.se/911bd22d1us,"About One Acre FundFounded in 2006, One Acre F..."
54,1005 times,"Kigali , RW","Agriculture, Business, Management, Procuremen...",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Wednesday, 30/11/2022 23",1,One Acre Fund,Rwanda Procurement Specialist,https://grnh.se/89c963c51us,"About One Acre FundFounded in 2006, One Acre F..."
55,762 times,", RW",Agriculture,High school,Entry level (1 to 3 years of experience),Full-time,"Monday, 28/11/2022 12",2,Kivu Choice Ltd,Security Officer,https://www.jobinrwanda.com/form/default-job-a...,Job Title: Security OfficerLocation: Kigembe/M...
56,731 times,", RW","Agriculture, Other",None,Mid career (3 to 5 years of experience),Full-time,"Monday, 28/11/2022 00",1,Kivu Choice Ltd,Security Supervisor,https://www.jobinrwanda.com/form/default-job-a...,Job Title: Security SupervisorLocation: Kigemb...
57,733 times,", RW","Computer and IT, Education, Other",Bachelor,Not specified,Full-time,"Monday, 28/11/2022 23",1,University of Global Health Equity (UGHE),Instructional Technology Assistant,https://www.pih.org/pages/employment?p=job%2Fo...,DescriptionJob Title:Instructional Technology ...
58,130 times,", RW","Medicine, Other",PhD,Senior (5+ years of experience),Full-time,"Monday, 28/11/2022 23",1,University of Global Health Equity (UGHE),Associate Professor and Head of Human Anatomy,https://www.pih.org/pages/employment?p=job%2Fo...,Job Title:Associate ProfessorandHead of Human ...
59,365 times,"Kirehe , RW","Agriculture, Agronomy, Business, Demography a...",Bachelor,Entry level (1 to 3 years of experience),Full-time,"Saturday, 26/11/2022 23",1,One Acre Fund,Rwanda Potato Seed Store Manager,https://grnh.se/03d933001us,"About One Acre FundFounded in 2006, One Acre F..."
60,647 times,", RW","Agriculture, Food sciences, Other",Bachelor,Senior (5+ years of experience),Full-time,"Friday, 18/11/2022 17",2,University of Rwanda Holdings Group Limited (U...,"Agronomist Technician, G-2(C)",https://www.jobinrwanda.com/form/default-job-a...,VACANCY POSITIONS AT UR HG LTD ANNOUNCEMENTUni...
61,181 times,", RW","Health, Medicine, Other",Bachelor,Entry level (1 to 3 years of experience),Full-time,"Friday, 18/11/2022 17",2,University of Rwanda Holdings Group Limited (U...,Dental Doctors /Dental Surgeon,https://www.jobinrwanda.com/form/default-job-a...,VACANCY POSITIONS AT UR HG LTD ANNOUNCEMENTUni...
62,697 times,", RW","Business, Management, Other, Public health, S...",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Monday, 14/11/2022 23",1,Gahini District Hospital,Head of Rehabilitation Department,https://www.jobinrwanda.com/form/default-job-a...,JOB ADVERTISEMENTJob Title: Head of Rehabilita...


In [15]:
df['Is IT Job'] = False

In [ ]:

def isIT_Job():
  for index in range(len(df)):
    # print("row: ", "Computer" in row[2] or "IT" in row[2])
      if "Computer" in df.iloc[index]['Sector'] or "IT" in df.iloc[index]['Sector']:
        df['Is IT Job'][index] = True

isIT_Job()

In [ ]:
df[df['Is IT Job'] == True]

In [ ]:
# DATA PRESENTATION
ITkeywords = ["Computer and IT", ""]

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **TRANSLATOR SECTION** %%%%%%%%%%%%%%%%%%%%%

In [ ]:
class translator:
  def __init__(self):
    # INITIALIZE THE MODEL FOR TRANSLATE
    self.model = EasyNMT("opus-mt")
  
  def supportedLangs(self, all_langs, supported):
    with open(all_langs) as f_json:
      content = f_json.read()
    langs = json.loads(content)

    with open(supported) as f2_json:
          content = f2_json.read()
    supplangs = json.loads(content)
    supp_codes = set(list(langs.values())).intersection(set(supplangs))
    supp_keyValue = {}
    for key in langs.keys():
      if langs[key] in supp_codes:
        supp_keyValue[key] = langs[key]
    return supp_keyValue
    

  def translate(self, sentences, targetLang, source = 'en', single = True):
    fail_message = "[FAIL]: destination lang is not a suported language by the model"
    success = 0
    if single:
      try:
        translation = self.model.translate(sentences, source_lang = source, target_lang = targetLang)
        return translation
      except:
        print(fail_message)
    else:
      translated = []
      for target in targetLang:
        try:
          translation = self.model.translate(sentences, source_lang = source, target_lang = target)
          translated.append(translation)
          success += 1
        except:
          print(fail_message)
      return translated, success


nest_asyncio : helps to make fastapi asynchronous

In [ ]:
# draft for consultancy
link = "https://www.jobinrwanda.com/jobs/consultancy"
soups = []
tags_contents = [[(child,soups.append(soup)) for child in final_div.children] for soup in [bs(requests.get(link).content, "html.parser") for link in ["https://jobinrwanda.com"+a['href'] 
        for div in soup.find_all('div', class_ = "card-body p-2") for a in div.find_all('a') if "job" in a['href']]]
         for final_div in soup.find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
         ]







         